In [32]:
import json
import os
import pandas
import pyarrow
import sys

from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
import prepare_data
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
api_keys_for_analysis_query = create_queries.create_api_keys_for_analysis_query()
api_keys_for_analysis_dataframe = pandas.read_sql_query(api_keys_for_analysis_query, engine)

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 400 Name is_deleted not found inside metrc_api_keys at [13:44]

(job ID: a3e0bdca-bd82-4b3a-8551-c9ca324108bd)

                    -----Query Job SQL Follows-----                     

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:		select
   3:			companies.identifier,
   4:			companies.name,
   5:			metrc_api_keys.us_state,
   6:			metrc_api_keys.is_functioning,
   7:			metrc_api_keys.facilities_payload
   8:		from
   9:			metrc_api_keys
  10:			inner join companies on metrc_api_keys.company_id = companies.id
  11:		where
  12:			True
  13:			and metrc_api_keys.is_deleted != True
  14:		order by
  15:			metrc_api_keys.last_used_at desc
  16:	
    |    .    |    .    |    .    |    .    |    .    |    .    |
[SQL: 
		select
			companies.identifier,
			companies.name,
			metrc_api_keys.us_state,
			metrc_api_keys.is_functioning,
			metrc_api_keys.facilities_payload
		from
			metrc_api_k

In [ ]:
api_keys_dataframe = api_keys_for_analysis_dataframe

In [ ]:
api_key_records = api_keys_dataframe.to_dict('records')

In [11]:
api_key_records[0]

{'identifier': 'UR',
 'name': 'Uprooted (San Ysidro)',
 'us_state': 'CA',
 'is_functioning': True,
 'facilities_payload': {'facilities': '[{"HireDate":"2020-01-02","IsOwner":false,"IsManager":false,"Occupations":[],"Name":"680 BROADWAY MASTER LLC","Alias":null,"DisplayName":"680 BROADWAY MASTER LLC","CredentialedDate":"2019-10-31","SupportActivationDate":null,"SupportExpirationDate":null,"SupportLastPaidDate":null,"FacilityType":{"IsMedical":true,"IsRetail":true,"IsHemp":false,"RestrictHarvestPlantRestoreTimeHours":null,"TotalMemberPatientsAllowed":null,"CanGrowPlants":false,"CanCreateOpeningBalancePlantBatches":false,"CanTagPlantBatches":false,"CanAssignLocationsToPlantBatches":false,"PlantsRequirePatientAffiliation":false,"PlantBatchesCanContainMotherPlants":false,"CanUpdatePlantStrains":false,"CanTrackVegetativePlants":false,"CanCreateImmaturePlantPackagesFromPlants":false,"CanPackageVegetativePlants":false,"CanPackageWaste":false,"CanReportHarvestSchedules":false,"CanSubmitHarvests

In [31]:
for api_key_record in api_key_records:
    company_name = api_key_record['name']
    company_identifier = api_key_record['identifier']
    print(f'Processing API key for company: {company_name} {company_identifier}')
    facilities_payload = api_key_record['facilities_payload']
    if facilities_payload:    
        facilities_json = json.loads(facilities_payload['facilities'])
        for facility_json in facilities_json:
            facility_name = facility_json['Name']
            facility_display_name = facility_json['DisplayName']
            print(f'{facility_name} ({facility_display_name})')
            license_number = facility_json['License']['Number']
            license_type = facility_json['License']['LicenseType']
            print(f'{license_number} ({license_type})')
    else:
        print('No facilities data')

    print('---')
    print('---')

Processing API key for company: Uprooted (San Ysidro) UR
680 BROADWAY MASTER LLC (680 BROADWAY MASTER LLC)
C10-0000590-LIC (Cannabis - Retailer License)
BANANA LLC (BANANA LLC)
C10-0000618-LIC (Cannabis - Retailer License)
CALGEN TRADING INC. (CALGEN TRADING INC.)
C10-0000776-LIC (Cannabis - Retailer License)
UL SAN JOSE LLC (UL SAN JOSE LLC)
C12-0000333-LIC (Cannabis - Microbusiness License)
ULBP Inc. (ULBP Inc.)
C11-0000678-LIC (Cannabis - Distributor License)
ULBP, INC. (ULBP, INC.)
C10-0000246-LIC (Cannabis - Retailer License)
UPROOTED LM LLC (UPROOTED LM LLC)
C10-0000731-LIC (Cannabis - Retailer License)
UPROOTED, INC. (UPROOTED, INC.)
C10-0000464-LIC (Cannabis - Retailer License)
---
---
Processing API key for company: Greenleaf GP
Greenleaf Processors, Inc. (Greenleaf Processors, Inc.)
CCL19-0005288 (A-Processor)
Greenleaf Processors, Inc. (Greenleaf Processors, Inc.)
CDPH-10002016 (AM-Type 7: Volatile Solvent Extraction)
GREENLEAF PROCESSORS, INC. (GREENLEAF PROCESSORS, INC.)
C